In [6]:
import wfdb
import math
from wfdb import processing
file_destination=r"D:\vf predict\cu-ventricular-tachyarrhythmia-database-1.0.0"+"\cu01"

def mean(file_destination):
    sig, fields = wfdb.rdsamp(file_destination , channels=[0] ,sampto=40000 )
    xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
    xqrs.detect()
    print(fields)
    wfdb.plot_items(signal=sig, ann_samp=[xqrs.qrs_inds] )

    #sqrs.qrs_inds is array containing indexes of qrs ?
    rpeaksum=0
    for i in xqrs.qrs_inds:
        rpeaksum+=sig[i]
    
    rmean=(rpeaksum/len(xqrs.qrs_inds))
    print("mean is",rmean)

#finding standard deviation
rstdsum=0
for i in xqrs.qrs_inds:
    rstdsum+=(sig[i]-rmean)**2
rstd=math.sqrt(rstdsum/len(xqrs.qrs_inds))
print("standard deviation is",rstd)

standard deviation is 0.2339456407036963


Area??